In [6]:
from nebula_api.milvus_api import MilvusAPI
import torch
import heapq
#import cv2
from nebula_api.nebula_enrichment_api import NRE_API

#from nebula_api.mdmmt_api.mdmmt_api import MDMMT_API
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.vlmapi import VLM_API

In [7]:
class VCOMET_ACTIONS:
    def __init__(self):
        self.milvus_actions = MilvusAPI(
            'milvus', 'vcomet_vit_embedded_actions', 'nebula_visualcomet', 768)
        self.milvus_places = MilvusAPI(
            'milvus', 'vcomet_vit_embedded_place', 'nebula_visualcomet', 768)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.nre = NRE_API()
        self.db = self.nre.db
        self.places_collection = self.db.collection("nebula_vcomet_places_playground")
        #self.gdb = self.nre.gdb
        self.clipmodel = VLM_API(model_name='clip_vit')
        self.mdmmtmodel = VLM_API()
    
    def get_playground_movies(self):
        return(['Movies/114206816', 'Movies/114206849', 'Movies/114206892', 'Movies/114206952', 'Movies/114206999', 'Movies/114207139', 'Movies/114207205', 'Movies/114207240', 'Movies/114207265', 'Movies/114207324', 'Movies/114207361', 'Movies/114207398', 'Movies/114207441', 'Movies/114207474', 'Movies/114207499', 'Movies/114207550', 'Movies/114207668', 'Movies/114207740', 'Movies/114207781', 'Movies/114207810', 'Movies/114207839', 'Movies/114207908', 'Movies/114207953', 'Movies/114207984', 'Movies/114208064', 'Movies/114208149', 'Movies/114208196', 'Movies/114208338', 'Movies/114208367', 'Movies/114208576', 'Movies/114208637', 'Movies/114208744', 'Movies/114208777', 'Movies/114208820', 'Movies/114206358', 'Movies/114206264', 'Movies/114206337', 'Movies/114206397', 'Movies/114206632', 'Movies/114206597', 'Movies/114206691', 'Movies/114206789', 'Movies/114207184', 'Movies/114206548'])
        
    def get_playground_movies_and_scenes(self):
        return([('Movies/114206952', 1),
                ('Movies/114207324', 0),
                ('Movies/114207908', 0),
                ('Movies/114208149', 2),
                ('Movies/114208196', 1),
                ('Movies/114208338', 0),
                ('Movies/114208576', 0),
                ('Movies/114208744', 0),
                ('Movies/114208744', 2),
                ('Movies/114206337', 0),
                ('Movies/114206337', 1),
                ('Movies/114206548', 0),
                ('Movies/114206548', 1)])

    def get_actions_for_scene(self, movie, stage):
        #movie_candidates = []
        vectors = []
        candidates_actions = []
        scored_actions = {}
        top_actions = []
        #print("Find candidates for scene")
        #input()
        path = ""  
        url_prefix = "http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/"
        url = self.nre.get_movie_url(movie)          
        clip_v = self.clipmodel.encode_video(movie, stage, class_name='clip_vit' )
        mdmmt_v = self.mdmmtmodel.encode_video(movie, stage)

        vectors.append(clip_v.tolist()[0])
        if clip_v is not None:
            clip_v = clip_v.tolist()[0]
            similar_nodes = self.milvus_actions.search_vector(50, clip_v)
            #max_sim = similar_nodes[0][0]
            #print("Candidate Places of scene")
            for node in similar_nodes:
                #stage_candidates_actions.append([node[0], node[1]['sentence']])
                candidates_actions.append(node[1]['sentence'])
            mdmmvtext_v = self.mdmmtmodel.encode_text(candidates_actions)
            scores = torch.matmul(mdmmvtext_v, mdmmt_v)
            for txt, score in zip(candidates_actions, scores):
                scored_actions[score.tolist()]= txt
            top_k = heapq.nlargest(5, scored_actions)
            for k in top_k:
                top_actions.append(scored_actions[k])
        movie_candidates = top_actions
        return(movie_candidates)

    def get_places_for_scene(self, movie, stage):
        #movie_candidates = []
        vectors = []
        candidates_actions = []
        scored_actions = {}
        top_actions = []
        #print("Find candidates for scene")
        #input()
        path = ""  
        url_prefix = "http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/"
        url = self.nre.get_movie_url(movie)          
        clip_v = self.clipmodel.encode_video(movie, stage, class_name='clip_vit' )
        mdmmt_v = self.mdmmtmodel.encode_video(movie, stage)

        vectors.append(clip_v.tolist()[0])
        if clip_v is not None:
            clip_v = clip_v.tolist()[0]
            similar_nodes = self.milvus_places.search_vector(50, clip_v)
            #max_sim = similar_nodes[0][0]
            #print("Candidate Places of scene")
            for node in similar_nodes:
                #stage_candidates_actions.append([node[0], node[1]['sentence']])
                candidates_actions.append(node[1]['sentence'])
            mdmmvtext_v = self.mdmmtmodel.encode_text(candidates_actions)
            scores = torch.matmul(mdmmvtext_v, mdmmt_v)
            for txt, score in zip(candidates_actions, scores):
                scored_actions[score.tolist()]= txt
            top_k = heapq.nlargest(5, scored_actions)
            for k in top_k:
                top_actions.append(scored_actions[k])
             
        movie_candidates = top_actions
        return(movie_candidates)

In [8]:
def get_lsmdc_s1(db):
    s1_lsmdc_movies = []
    
    query = 'FOR doc IN s1_lsmdc_dima RETURN doc'
   
    cursor = db.aql.execute(query)
    for data in cursor:
        s1_lsmdc_movies.append(data)
    #print(s1_lsmdc_movies)
    return(s1_lsmdc_movies)

In [9]:
kg = VCOMET_ACTIONS()
db = kg.nre.db
print(db)

Milvus Collection Loaded:  vcomet_vit_embedded_actions
Milvus Collection Loaded:  vcomet_vit_embedded_place
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Available class names: ['clip_vit', 'clip_rn', 'mdmmt_max', 'mdmmt_mean', 'mdmmt_legacy']
INFO:tensorflow:Restoring parameters from /storage/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.d

Available class names: ['clip_vit', 'clip_rn', 'mdmmt_max', 'mdmmt_mean', 'mdmmt_legacy']
<StandardDatabase nebula_playground>


In [10]:
def update_actions(db, actions, movie_id, scene_element):
    query = 'FOR doc IN s1_lsmdc_dima FILTER doc.movie_id == @movie_id AND  doc.scene_element == @scene_element UPDATE doc WITH {actions: @actions} in s1_lsmdc_dima'
    db.aql.execute(query, bind_vars={'actions': actions, 'movie_id': movie_id, 'scene_element': scene_element})

def update_places(db, places, movie_id, scene_element):
    query = 'FOR doc IN s1_lsmdc_dima FILTER doc.movie_id == @movie_id AND  doc.scene_element == @scene_element UPDATE doc WITH {places: @places} in s1_lsmdc_dima'
    db.aql.execute(query, bind_vars={'places': places, 'movie_id': movie_id, 'scene_element': scene_element})

In [11]:


s1_movies = get_lsmdc_s1(db)

In [12]:
print(s1_movies)

[{'_key': '135886515', '_id': 's1_lsmdc_dima/135886515', '_rev': '_d1N1JeW---', 'ref': 'Lord of the Rings', 'movie_id': 'Movies/114207908', 'scene_element': 0, 'url': 'http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946.mp4', 'base': ['A man in a suit and a woman in a dress'], 'actions': ['fellowship', 'get his army to gondor', 'remember his oath to protect the king', 'decide that he would fight for his king', 'kill orcs with his sword'], 'places': ['in a fantasy setting', 'at an encampment', 'in a time of war', 'among a battle with a castle behind everyone', 'at the gates of gondor'], 'experts': {'warrior': ['metal helmet', 'shields', 'crown', 'knight', 'bearded'], 'soldiers': ['three', 'weapons', 'spears'], 'back of a lady': ['blonde', 'back']}, 'groundtruth': ['A blue-eyed bearded warrior with a metal helmet and shield puts his arm on a blonde lady with three armed soldiers i

In [13]:


movies = kg.get_playground_movies()
for i, movie in enumerate(s1_movies):
    places = kg.get_places_for_scene(movie['movie_id'], movie['scene_element'])
    print("Places - Movie: ", i)
    print(places)
    update_places(db, places, movie['movie_id'], movie['scene_element'])
    #kg.places_collection.insert(places)
    


http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946.mp4
Movie info: {'arango_id': 'Movies/114207908', 'description': '0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f9305a676703419bb958d2c44454293f', 'mdfs': [[2, 57, 112], [116, 122, 129]], 'scene_elements': [[0, 114], [114, 131]]}
fn path: /tmp/video_file.mp4
/tmp/video_file.mp4
Scene:  0
http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946.mp4
Movie info: {'arango_id': 'Movies/114207908', 'description': '0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f9305a676703419bb958d2c44454293f', 'mdfs': [[2, 57, 112], [116, 1

In [ ]:
for i, movie in enumerate(s1_movies):
    #scene_elements = kg.nre.get_stages(movie)
    #
    print("Actions - Movie: ", i)
    actions = kg.get_actions_for_scene(movie['movie_id'], movie['scene_element'])
    print(actions)
    update_actions(db, actions, movie['movie_id'], movie['scene_element'])
    

Actions - Movie:  0
http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946.mp4
Movie info: {'arango_id': 'Movies/114207908', 'description': '0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f9305a676703419bb958d2c44454293f', 'mdfs': [[2, 57, 112], [116, 122, 129]], 'scene_elements': [[0, 114], [114, 131]]}
fn path: /tmp/video_file.mp4
/tmp/video_file.mp4
Scene:  0
http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946.mp4
Movie info: {'arango_id': 'Movies/114207908', 'description': '0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f9305a676703419bb958d2c44454293f', 'mdfs': [[